In [150]:
import numpy as np
from numpy.linalg import det, inv, eigvals, norm, matrix_rank
import scipy as sp
from scipy.linalg import null_space, sqrtm

np.set_printoptions(precision=2)

Real finite collection of vectors:<br>
$$F=(f_i)_{i\in I}\subset \mathbb{R}^n$$
$$I=\{1,...,m\}$$
$$m/n\; \dots \text{ "redundancy"}$$
$F$ is a frame if there are $A,B>0$ such that $$A\cdot \|f\|^2 \leq \sum_{i\in I} \vert \langle f,f_i \rangle \vert^2 \leq B\cdot \|f\|^2$$
for all $f\in \mathbb{R}^n$. This is equivalent to $F$ being a spanning set for $\mathbb{R}^n$ and $A,B$ indicate the numerical stability (we'll see). Clearly, $ m\geq n$ has to hold. <br>

Today: Real random frames:<br>
$$f_i \sim \mathcal{N}(0,1_n)\; \text{i.i.d. for all } i\in I$$

In [166]:
# dimensions
m = 6
n = 3

Analysis operator maps a vector $f\in \mathbb{R}^n$ to its frame coefficients via inner products with the $f_i$.
$\begin{align}
    T^*:\mathbb{R}^n &\rightarrow \mathbb{R}^m \\ f&\mapsto\left(\langle f, f_i\rangle\right)_{i\in I} = \left(\langle f_i, f\rangle\right)_{i\in I}
\end{align}$
Matrix multiplication from the left:
$$T^* f = \begin{pmatrix}
    -\quad  f_1\quad  -\\
  
    -\quad  f_2\quad  -\\
      \vdots  \\
    -\quad  f_m\quad  -
\end{pmatrix} f$$
If $F$ is a frame, $T^*$ is injective iff $T^*$ has full (column) rank.

Random: $T^*$ is a Gaussian matrix, which has full rank with probability one, i.e., a random collections of random vectors is a frame with prob one.

In [167]:
T_ana = np.random.randn(m, n)
print('T* =')
print(T_ana)
print('rank =', matrix_rank(T_ana))

f = np.random.randn(n)
Tf = T_ana @ f
print('frame coeff:', Tf)
# "change of basis"

T* =
[[ 0.07 -0.04  1.26]
 [-1.41 -0.94 -1.39]
 [-0.76 -1.05  0.55]
 [-1.72 -0.67 -0.8 ]
 [ 0.71  1.68 -0.74]
 [ 0.39 -0.01 -1.72]]
rank = 3
frame coeff: [ 0.63  0.65  1.07  1.3  -1.11 -1.37]


Synthesis operator maps frame coefficients $c = (c_i)_{i\in I}$ back to the signal space $\mathbb{R}^n$ via linear combinations of $f_i$.
\begin{align}
    T:\mathbb{R}^m &\rightarrow \mathbb{R}^n\\
    (c_i)_{i\in I}&\mapsto \sum_{i\in I} c_i\cdot f_i.
\end{align}
Matrix multiplication from the left:
$$T c = (T^*)^\top c = \begin{pmatrix}
    \vert & \vert & & \vert\\
    f_1 & f_2 & \cdots & f_m\\
    \vert & \vert & & \vert\\
\end{pmatrix} c$$
If $F$ is a frame, $T$ is surjective iff $T$ has full (row) rank.

In [168]:
T_syn = T_ana.T
print('T =')
print(T_syn)
print('rank =', matrix_rank(T_syn))

c = np.random.randn(m)
Tc = T_syn @ c
print('vector assoc with frame coefficients c:', Tc)

T =
[[ 0.07 -1.41 -0.76 -1.72  0.71  0.39]
 [-0.04 -0.94 -1.05 -0.67  1.68 -0.01]
 [ 1.26 -1.39  0.55 -0.8  -0.74 -1.72]]
rank = 3
vector assoc with frame coefficients c: [ 1.29  1.36 -4.68]


Gramian does synthesis, followed by analysis and gives the cross-correlations between the frame elements.
\begin{align}
    G:\mathbb{R}^m &\rightarrow \mathbb{R}^m\\
    c&\mapsto \left(\left\langle \sum_{j\in I} c_j f_j,f_i \right\rangle\right)_{i\in I}.
\end{align}
As a matrix:
$$G=T^*T$$
$$G[i,j] = \langle f_i,f_j \rangle$$
$G$ is
- injective from $R_{T^*}$ to $R_{T^*}$ but not invertible on $\mathbb{R}^m$!
- self-adjoint
- the non-zero eigenvalues agree with those of $S$

In [169]:
G = T_ana @ T_syn
print('Gram =')
print(G)
print('det:',det(G))
print('self-adjoint:',np.all(G.T == G))

lam_G = eigvals(G)
#print(lam_G)
lam_pos = [l for l in lam_G if abs(l)>1e-10]
print('non-zero eigenvalues of G:')
print(np.real(lam_pos))
# whats the main diagonal of G?

Gram =
[[ 1.58 -1.81  0.68 -1.11 -0.94 -2.14]
 [-1.81  4.82  1.3   4.19 -1.57  1.85]
 [ 0.68  1.3   1.99  1.58 -2.71 -1.25]
 [-1.11  4.19  1.58  4.07 -1.77  0.72]
 [-0.94 -1.57 -2.71 -1.77  3.86  1.54]
 [-2.14  1.85 -1.25  0.72  1.54  3.12]]
det: 0.0
self-adjoint: True
non-zero eigenvalues of G:
[10.91  7.53  1.  ]


In [170]:
# the norms squared are on the diagonal
print(np.linalg.norm(T_ana[0,:])**2)

1.5847216366792887


Frame operator does analysis, followed by synthesis.
\begin{align}
    S:\mathbb{R}^n &\rightarrow \mathbb{R}^n\\
    f&\mapsto \sum_{i\in I} \langle f,f_i \rangle\cdot f_i.
\end{align}
As matrix:
$$S = TT^*$$
$S$ is
- invertible
- self-adjoint
- positive

In [171]:
S = T_syn @ T_ana
print('S =')
print(S)
print('determinant:',det(S))
print('self-adjoint:',np.all(S.T == S))
print('positive:',np.dot(S @ f,f))

lam = eigvals(S)
print('eigenvalues of S:')
print(lam)

S =
[[ 6.23e+00  4.49e+00  1.82e+00]
 [ 4.49e+00  5.25e+00 -4.93e-04]
 [ 1.82e+00 -4.93e-04  7.98e+00]]
determinant: 82.5153138649509
self-adjoint: True
positive: 6.77610598880911
eigenvalues of S:
[ 1.   10.91  7.53]


The optimal frame bounds $A,B$ of $F$ are given by the smallest and largest eigenvalues of $S$.<br>
$B^{-1},A^{-1}$ are the smallest and largest eigenvalues of $S^{-1}$.

In [172]:
A = min(lam)
B = max(lam)
print('frame bounds:', A,B)

S_inv = inv(S)
lam_inv = eigvals(S_inv)
A_inv = min(lam_inv)
B_inv = max(lam_inv)
print('frame bounds of inverse:')
print(A_inv,B_inv)
print('inverse of frame bounds:')
print(B**(-1), A**(-1))

frame bounds: 1.0035151655956707 10.913376474449668
frame bounds of inverse:
0.09163067015430017 0.9964971475108879
inverse of frame bounds:
0.09163067015430046 0.9964971475108858


Canonical dual frame $\tilde{F}=(S^{-1}f_i)_{i\in I}$

In [173]:
# canonical dual frame
T_dual_syn = S_inv @ T_syn
print('synthesis operator:')
print(T_dual_syn)
print('check duality:')
S_mixed = T_dual_syn @ T_ana
print(S_mixed)
# this shows that every frame is dual to its dual

synthesis operator:
[[-0.09 -0.15  0.   -0.49 -0.28  0.4 ]
 [ 0.07 -0.05 -0.2   0.29  0.56 -0.34]
 [ 0.18 -0.14  0.07  0.01 -0.03 -0.31]]
check duality:
[[ 1.00e+00  3.04e-16 -1.05e-16]
 [-4.84e-16  1.00e+00 -7.51e-17]
 [-1.66e-16 -8.90e-17  1.00e+00]]


check that $B^{-1},A^{-1}$ are the framebounds of the canonical dual frame

In [174]:
S_can = T_dual_syn @ T_dual_syn.T
lam_can = eigvals(S_can)
A_can = min(lam_can)
B_can = max(lam_can)
print(A_can, B_can)
print(B**(-1), A**(-1))

0.09163067015430017 0.9964971475108879
0.09163067015430046 0.9964971475108858


Non-canonical dual frame: $\tilde{F}'=(S^{-1}f_i + \delta_i)_{i\in I}$ for $\delta_i \in \mathcal{N}_T$ (add elements form the null-space of the sythesis operator)

In [175]:
# ONB for the null-space of T_syn
B = null_space(T_syn)
f_kernel = B[:,0].reshape(1,m)
print('element in the kernel of F_syn:', f_kernel)
print('check:')
print(T_syn @ f_kernel.T)

element in the kernel of F_syn: [[ 0.24 -0.56 -0.29  0.58 -0.26  0.38]]
check:
[[-2.78e-17]
 [-1.94e-16]
 [ 0.00e+00]]


In [177]:
T_dual2_syn = T_dual_syn + f_kernel
print('synthesis operator:')
print(T_dual2_syn)
print('check duality:')
print(T_dual2_syn @ T_ana)
# check that f_kernel is orthogonal to the range of T_ana
print('Since Tf_kern = 0, so is f_kernT*:')
print(f_kernel @ T_ana)

synthesis operator:
[[ 0.14 -0.71 -0.29  0.09 -0.54  0.77]
 [ 0.31 -0.62 -0.49  0.87  0.3   0.03]
 [ 0.41 -0.71 -0.22  0.59 -0.29  0.07]]
check duality:
[[ 1.00e+00 -1.66e-16 -3.46e-17]
 [-3.85e-16  1.00e+00  2.34e-17]
 [-3.39e-16 -3.73e-16  1.00e+00]]
Since Tf_kern = 0, so is f_kernT*:
[[-2.78e-17 -1.94e-16  0.00e+00]]


Another non-canonical dual: sparse! choose a sub-frame of $F$ and compute the canonical dual for this one.

In [126]:
# choose 3 random frame elements
idx = np.random.randint(0, m, 3)
print(np.sort(idx))

[1 2 4]


In [127]:
S_small = T_syn[:,idx] @ T_ana[idx,:]
S_small_inv = inv(S_small)
T_dual3_small_syn = S_small_inv @ T_syn[:,idx]
T_dual3_syn = np.zeros((n,m))
T_dual3_syn[:,idx] = T_dual3_small_syn
print('synthesis operator:')
print(T_dual3_syn)
print('check duality:')
print(T_dual3_syn @ T_ana)

synthesis operator:
[[ 0.   -0.52  1.15  0.   -0.67  0.  ]
 [ 0.   -0.37 -0.06  0.   -0.44  0.  ]
 [ 0.    0.3   0.17  0.   -0.12  0.  ]]
check duality:
[[ 1.00e+00  1.86e-15  5.85e-15]
 [-7.11e-17  1.00e+00  1.02e-15]
 [ 1.71e-17  8.47e-17  1.00e+00]]


canonical dual has minimal $\ell^2$ norm

In [144]:
print('canonical:',norm(T_dual_syn))
print('non-canonical:',norm(T_dual2_syn))
print('non-canonical2:',norm(T_dual3_syn))

canonical: 1.5291732671117872
non-canonical: 1.845380292156322
non-canonical2: 1.5808459399700505


Canonical tight frame $F_t=(S^{-1/2}f_i)_{i\in I}$

In [141]:
# canonical tight frame
S_inv_sqrt = sqrtm(S_inv)
T_syn_tight = S_inv_sqrt @ T_syn
T_ana_tight = T_syn_tight.T
S_tight = T_syn_tight @ T_ana_tight
print('frame operator:')
print(S_tight)

frame operator:
[[ 1.00e+00 -8.44e-16  4.96e-16]
 [-8.44e-16  1.00e+00  1.21e-15]
 [ 4.96e-16  1.21e-15  1.00e+00]]


Redundant frames, which ones can we remove?
\begin{align}
\langle f_i, S^{-1}f_i \rangle &\neq 1 \rightarrow \text{can remove it}\\
\langle f_i, S^{-1}f_i \rangle &= 1 \rightarrow \text{incomplete} 
\end{align}
intuition: if $\langle f_i, S^{-1}f_i \rangle = 1$, then $f_i$ spans its own 1-D sub-space

In [129]:
T_ana_2 = np.random.randn(n, n)
f_add = (np.random.randn(1)*T_ana_2[0,:] + np.random.randn(1)*T_ana_2[1,:]).reshape(1,n)
T_ana_dep = np.append(T_ana_2, f_add, axis=0)
print(T_ana_dep)

[[ 1.53 -0.44  0.17]
 [ 0.85  1.45  1.26]
 [-1.5  -1.99  1.41]
 [-1.56 -1.9  -1.79]]


In [130]:
S_dep = T_ana_dep.T @ T_ana_dep
S_dep_inv = inv(S_dep)
T_dual_syn_dep = S_dep_inv @ T_ana_dep.T
print('check condition:')
print(np.diag(T_ana_dep @ T_dual_syn_dep))
#[np.dot(T_ana_dep[k,:], T_dual_syn_dep[:,k]) for k in range(n+1)]

check condition:
[0.98 0.36 1.   0.66]
